## Linear least squares

Here the goal is to show how a model fitting problem can be decomposed and solved by ADMM.

For this purpose, this notebook considers least-squares fitting.

#### Problem

minimize: $\frac{1}{2} \; \| A \; x - b \|_2^2$

### Steps

1. Devise a computation graph representing the problem as a bipartite graph
2. Implement nodes as Java classes extending org.admm4j.core.Node
3. Create the JSON input defining the graph
4. Execute admm4j
5. Import and analyze results

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import json
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

### Generate data for linear regression

In [2]:
np.random.seed(1234)

num_points = 40

x = np.array([-5, -4, -3, -2, -1, 1, 2, 3, 4, 5]) # underlying values 

A = -20 + 40 * np.random.rand(num_points, x.shape[0]) # features
noise = np.random.normal(0, 1, (num_points,x.shape[0])) # noise
b = np.dot(A+noise, x) # target values

### Step 1. Devise a computation graph representing the problem as a bipartite graph

The data is distributed across worker nodes.\
Each worker solves least squares problem using local data.\
Master averages the results collected from workers.

<img src="images/master-workers1.png" width="600" height="600" style="float: center"/>

### Step 2. Implement nodes as Java classes extending org.admm4j.core.Node

The following classes are implemented

1. org.admm4j.demo.ml.linearmodel.LeastSquaresNode.java

2. org.admm4j.demo.common.AveragingNode.java

### Step 3. Create the JSON input defining the graph

The JSON input has a well-defined structure allowing to define an arbitrary bipartite graph.

Here the JSON input is created in python.

In [3]:
# here we cluster the data into 3 clusters with data being distributed across 4 workers having 9 points in each node
num_workers = 4
points_per_node = 10

# init nodesI.
nodesI = []
for i in range(0, num_workers):
    # define node
    node = {}
    node['name'] = 'worker{}'.format(i)
    node['class'] = 'org.admm4j.demo.ml.linearmodel.LeastSquaresNode'
    node['neighbors'] = ['master']
    node['input'] = {'A': A[i*points_per_node:(i+1)*points_per_node:,:].tolist(),\
                     'b': b[i*points_per_node:(i+1)*points_per_node].tolist()}
    # add to the list of nodesI
    nodesI.append(node)

# init nodesII
nodesII = []
node = {}
node['name'] = 'master'
node['class'] = 'org.admm4j.demo.common.AveragingNode'
node['neighbors'] = None
node['input'] = None
nodesII.append(node)

# init whole json model
graph = {'nodesI': nodesI, 'nodesII': nodesII}

#### Show JSON model

In [4]:
print(json.dumps(graph))

{"nodesI": [{"name": "worker0", "class": "org.admm4j.demo.ml.linearmodel.LeastSquaresNode", "neighbors": ["master"], "input": {"A": [[-12.339221984844308, 4.884350841593275, -2.4908904397154217, 11.41434334855077, 11.199032324752139, -9.096295788694334, -8.941429794276132, 12.074887101400769, 18.325574147348206, 15.037305389683787], [-5.687309201685334, 0.039805020938349145, 7.338517406885451, 8.508081079316007, -5.189969808384202, 2.4478474426249974, 0.12332661231238973, -19.44926201637271, 10.913064864494963, 15.305647625444664], [-5.404560643945109, 4.615847137339749, -16.98475033428094, -5.2470397599210195, 17.325604079300867, 6.055125729063096, -4.111896890953833, 11.549205717629821, -7.326555113245149, 2.723946105042767], [14.765095582449035, -2.5530630441728235, 12.085905683206363, -14.249327019417418, 8.170438844733418, 8.183252327582903, -11.248315773036456, 16.9947051446226, -2.314369783832934, 16.372638358898904], [-17.607631088805924, -12.628516647447455, -18.10578884793939

#### Save JSON input file

In [5]:
filename = 'least_squares_input.json'
fout = open(filename, 'w')
json.dump(graph, fout, indent=4)
fout.close()

### Step 4. Execute admm4j

Following parameters are provided:
1. -input
2. -output
3. -nvar
4. -rho

#### Note: -nvar and -rho are provided in command line

In [6]:
!java -jar admm4j-demo/target/admm4j-demo-1.0-jar-with-dependencies.jar\
           -input least_squares_input.json\
           -output least_squares_output.json\
           -nvar 10\
           -rho 1

### Step 5. Import and analyze results

In [7]:
fin = open('least_squares_output.json', 'r')
res = json.loads(fin.read())
fin.close()

#### Show results and evaluate performance

In [8]:
x = res.get('nodesI')[0].get('variables').get('master')

print('Coefficients', x)

print('Mean squared error: %.2f' % mean_squared_error(b, np.dot(A,x)))

Coefficients [-4.795597453255772, -3.968367529875729, -2.6521910090508722, -2.2005638387809294, -0.9914573623761608, 1.1220004863876587, 2.0058960115138804, 2.8389483775574624, 4.185975599976671, 5.25831356764516]
Mean squared error: 61.18


It can be seen that reasonable results are obtained.

#### Comparing with LinearRegression from scikit-learn

In [9]:
model = LinearRegression(fit_intercept=True).fit(A, b)

#print(model.intercept_)
print('Coefficients', model.coef_)

print('Mean squared error: %.2f' % mean_squared_error(b, model.predict(A)))

Coefficients [-4.80284843 -3.96067251 -2.64496968 -2.18549362 -0.98656868  1.10584387
  2.009803    2.88621879  4.18509478  5.27464015]
Mean squared error: 60.30


The difference can be due to the choice of scaling parameter $\rho$.

## Example 2: Different structure of computation graph

The purpose of this example is to show that the computation graph can be defined in many different ways as long as its structure represents a bipartite graph.

<img src="images/masters-2.png" width="800" height="800" style="float: center"/>

In this graph:
- workers solve least squares problems using local data
- masters coordinate connected nodes
- connector connects masters

Both connector and masters implement simple averaging.

Similar modeling approach can be adopted to address more complex problems.

### Create the JSON input defining the graph

In [10]:
# here we cluster the data into 3 clusters with data being distributed across 4 workers having 9 points in each node
num_workers = 4
points_per_node = 10

# init nodesI.
nodesI = []

node = {}
node['name'] = 'connector'
node['class'] = 'org.admm4j.demo.common.AveragingNode'
node['neighbors'] = None
node['input'] = None
nodesI.append(node)

for i in range(0, num_workers):
    # define node
    node = {}
    node['name'] = 'worker{}'.format(i)
    node['class'] = 'org.admm4j.demo.ml.linearmodel.LeastSquaresNode'
    node['neighbors'] = None
    node['input'] = {'A': A[i*points_per_node:(i+1)*points_per_node:,:].tolist(),\
                     'b': b[i*points_per_node:(i+1)*points_per_node].tolist()}
    # add to the list of nodesI
    nodesI.append(node)

# init nodesII
nodesII = []

node = {}
node['name'] = 'master0'
node['class'] = 'org.admm4j.demo.common.AveragingNode'
node['neighbors'] = ['worker0', 'worker1', 'connector']
node['input'] = None
nodesII.append(node)

node = {}
node['name'] = 'master1'
node['class'] = 'org.admm4j.demo.common.AveragingNode'
node['neighbors'] = ['worker2', 'worker3', 'connector']
node['input'] = None
nodesII.append(node)

# init whole json model
graph = {'nodesI': nodesI, 'nodesII': nodesII}

#### Show JSON model of the graph

In [11]:
print(json.dumps(graph))

{"nodesI": [{"name": "connector", "class": "org.admm4j.demo.common.AveragingNode", "neighbors": null, "input": null}, {"name": "worker0", "class": "org.admm4j.demo.ml.linearmodel.LeastSquaresNode", "neighbors": null, "input": {"A": [[-12.339221984844308, 4.884350841593275, -2.4908904397154217, 11.41434334855077, 11.199032324752139, -9.096295788694334, -8.941429794276132, 12.074887101400769, 18.325574147348206, 15.037305389683787], [-5.687309201685334, 0.039805020938349145, 7.338517406885451, 8.508081079316007, -5.189969808384202, 2.4478474426249974, 0.12332661231238973, -19.44926201637271, 10.913064864494963, 15.305647625444664], [-5.404560643945109, 4.615847137339749, -16.98475033428094, -5.2470397599210195, 17.325604079300867, 6.055125729063096, -4.111896890953833, 11.549205717629821, -7.326555113245149, 2.723946105042767], [14.765095582449035, -2.5530630441728235, 12.085905683206363, -14.249327019417418, 8.170438844733418, 8.183252327582903, -11.248315773036456, 16.9947051446226, -2

#### Save JSON input file

In [12]:
filename = 'least_squares_input2.json'
fout = open(filename, 'w')
json.dump(graph, fout, indent=4)
fout.close()

#### Execute admm4j

Following parameters are provided:
1. -input
2. -output
3. -nvar
4. -rho

#### Note: -nvar and -rho are provided in command line

In [13]:
!java -jar admm4j-demo/target/admm4j-demo-1.0-jar-with-dependencies.jar\
           -input least_squares_input2.json\
           -output least_squares_output2.json\
           -nvar 10\
           -rho 1

#### Import and analyze results

In [14]:
fin = open('least_squares_output2.json', 'r')
res = json.loads(fin.read())
fin.close()

# all nodes converged to the same variable values
x = res.get('nodesII')[0].get('variables').get('worker0')

print('Coefficients', x)

print('Mean squared error: %.2f' % mean_squared_error(b, np.dot(A,x)))

Coefficients [-4.794264220736139, -3.966588460214021, -2.652229092031883, -2.1995418098782498, -0.9911065704421048, 1.1224337798368926, 2.004735160304724, 2.8373401876842315, 4.185473461295792, 5.256226347660851]
Mean squared error: 61.18


The results are similar to those obtained previously.